In [1]:
from urllib.parse import urlparse,parse_qsl,urljoin
from pymongo import MongoClient
import requests
from ipapi import location

In [2]:
with open('logs.txt') as f:
    logs=f.readlines()

In [21]:
def parse_log(log):
    rec={}
    rec["api_name"],_,rec["date"],rec["time"],rec["hashcode"],rec["log_type"],rec["ip"],url=log.split()

    t=urlparse(url)
    
    rec['url']={'scheme':t.scheme,
               'netloc':t.netloc,
               'path':t.path[1:-1 if t.path.endswith('/') else None],
               'query':{par:val for par,val in parse_qsl(t.query)}}
    rec['hashcode']=rec['hashcode'][1:-1]
    return rec

In [53]:
records=[parse_log(rec) for rec in logs]

In [26]:
def get_location_mapper(records,geoip_url='https://freegeoip.app/json/'):
    mapper=dict.fromkeys({rec['ip'] for rec in records})
    for i in mapper:
        resp=requests.get(urljoin(geoip_url,i)")
        if resp.status_code==200:
            country_name=resp.json()['country_name']
            mapper[i]= 'undefined' if country_name=='' else country_name
        else:
            raise Exception('Problem in ip location service',resp.status_code,resp.content)
    return mapper

In [52]:
mapp=get_location_mapper(records)

NameError: name 'get_location_mapper' is not defined

In [55]:
c=0
for i in mapp:
    if mapp[i]=='':
        c+=1

145

In [61]:
with MongoClient() as client:
    client.itis.logs.insert_many(records)

In [59]:
for i in records:
    i['country']= mapp[i['ip']] if mapp[i['ip']]!='' else 'undefined'

In [60]:
records

[{'api_name': 'shop_api',
  'date': '2018-08-01',
  'time': '00:01:35',
  'hashcode': 'YQ4WUDJV',
  'log_type': 'INFO:',
  'ip': '121.165.118.201',
  'url': {'scheme': 'https',
   'netloc': 'all_to_the_bottom.com',
   'path': '',
   'query': {}},
  'country': 'South Korea'},
 {'api_name': 'shop_api',
  'date': '2018-08-01',
  'time': '00:01:47',
  'hashcode': 'E522JAEQ',
  'log_type': 'INFO:',
  'ip': '121.165.118.201',
  'url': {'scheme': 'https',
   'netloc': 'all_to_the_bottom.com',
   'path': 'fresh_fish',
   'query': {}},
  'country': 'South Korea'},
 {'api_name': 'shop_api',
  'date': '2018-08-01',
  'time': '00:03:02',
  'hashcode': 'BHWP66G8',
  'log_type': 'INFO:',
  'ip': '121.165.118.201',
  'url': {'scheme': 'https',
   'netloc': 'all_to_the_bottom.com',
   'path': '',
   'query': {}},
  'country': 'South Korea'},
 {'api_name': 'shop_api',
  'date': '2018-08-01',
  'time': '00:04:27',
  'hashcode': 'C0182Y6W',
  'log_type': 'INFO:',
  'ip': '121.165.118.201',
  'url': {'sch